In [1]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
import tqdm
import torch
import numpy as np
import pandas as pd
import plotly.express as px
import torch.optim as optim
from copy import deepcopy
from typing import Iterable

from model import LR
from data import FairnessDataset, Dataset, SyntheticDataset, GermanDataset, IncomeDataset, GermanPCADataset
from ei_effort import Optimal_Effort, PGD_Effort
from ei_utils import model_performance, pareto_frontier
from ei_model import EIModel, fair_batch_proxy, covariance_proxy

In [2]:
def generate_grid(center, width, n=15):
    if isinstance(width, int) or isinstance(width, float):
        width = [width for _ in range(len(center))]
    axes = [np.linspace(center[i]-width[i], center[i]+width[i], n) for i in range(len(center))]
    grids = np.meshgrid(*axes)
    points = np.stack([grid.reshape(-1) for grid in grids]).T
    return np.unique(points, axis=0)

In [3]:
def random_point(thetas):
    random_index = np.random.randint(len(thetas))
    return thetas[random_index]

In [4]:
def cache_result(cache, method, lamb, alpha, total_loss, pred_loss, fair_loss, theta, theta_adv, error):
    cache['method'].append(method)
    cache['lambda'].append(lamb)
    cache['alpha'].append(alpha)
    cache['total_loss'].append(total_loss)
    cache['pred_loss'].append(pred_loss)
    cache['fair_loss'].append(fair_loss)
    cache['theta'].append(theta)
    cache['theta_adv'].append(theta_adv)
    cache['error'].append(error)

In [5]:
torch.manual_seed(0)
# Instantiate the dataset
dataset = GermanPCADataset()

delta = dataset.delta
z_blind = True
tau = 0.5
# 1e-7, 0.25, 0.5, 0.75, 1-1e-7
# lamb = 1e-7
# lamb = .25
# lamb = .5
lamb = .75
# lamb = 1-1e-7
alpha = .5
proxy = covariance_proxy
effort = Optimal_Effort(dataset.delta)

# Train, Val, Test Split of Data
train_tensors, val_tensors, test_tensors = dataset.tensor(fold=0, z_blind=z_blind)
train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)

# Loss and model definition
loss_fn = torch.nn.BCELoss(reduction='mean')
model = LR(num_features=train_dataset.X.shape[1])
model_adv = LR(num_features=train_dataset.X.shape[1])

# Variables to track 
results = {'method': [], 'lambda': [], 'alpha': [], 'total_loss': [], 'pred_loss': [], 'fair_loss': [], 'theta': [], 'theta_adv': [], 'error': []}

       PCA1      PCA2
0  1.865175  0.563667
1  2.124531  2.106108
2 -0.716061 -1.760759
3 -1.496801  2.673849
4  0.101699 -1.301354


AttributeError: 'PCAGermanDataset' object has no attribute 'num_feats'